<a href="https://colab.research.google.com/github/KhannaShivang/BrainTumorDetection/blob/main/BrainTumorDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

--2024-07-03 19:00:42--  https://www.dropbox.com/s/dtw18saps0231yn/BrainTumorData.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/dtw18saps0231yn/BrainTumorData.zip [following]
--2024-07-03 19:00:43--  https://www.dropbox.com/s/raw/dtw18saps0231yn/BrainTumorData.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2024-07-03 19:00:43 ERROR 404: Not Found.



In [ ]:
#get the data from
!wget https://www.dropbox.com/s/dtw18saps0231yn/BrainTumorData.zip?dl=0

In [ ]:
import numpy
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob


In [ ]:
#count the numer of images in respective classes BrainTumor-0 healthy-1
ROOT_DIR=
number_of_imagaes={}

for dir in os.listdir(ROOT_DIR):
  number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))

number_of_images.items()


# WE WILL SPLIT THE DATA SUCH THAT
  *70% FOR TRAIN DATA

  *15% FOR VALIDATION


  *15% FOR TESTING

In [ ]:
def dataFolder(path,split):
  # WE WILL CREATE A TRAINGIN FOLDER
  if not os.path.exists("./"+path):
    os.mkdir("./"+path)

    for dir in os.listdir(ROOT_DIR):
      os.mkdirs("./"+path+"/"+dir)
      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR,dir)),
                                  size =(math.floor(split*number_of_images[dir])-5),replace=False):
        O=os.path.join(ROOT_DIR,dir,img)
        D=os.path.join("./"+path,dir)
        shutil.copy()<D
        os.remove(O)
  else:
    print("the folder exists")

In [ ]:
dataFolder("train",0.7)

In [ ]:
dataFolder("val",0.15)

In [ ]:
datafolder("test",0.15)

#Model Build

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

import keras

In [ ]:
#CNN model
model = Sequential()
model.add(Conv2D(filters=16 , kernel_size=(3,3),activation='relu' , input_shape = (224,224,3)))

model.add(Conv2D(filters=36 , kernel_size=(3,3),activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64 , kernel_size=(3,3),activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128 , kernel_size=(3,3),activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation= 'relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units= 1,activation= 'sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 conv2d_3 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 36)      0         
 D)                                                              
                                                                 
 conv2d_4 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)      

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

Preparing our data using Data Generator

In [ ]:
def preprocessingImages1(path):
  #input: Path
  #output: Pre processed images
  image_data=ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale=1/255, horizontal_flips= True) #Data Augmentation
  image=image_data.flow_from_directory(directory= path, target_size=(224,224), batch_size = 32,class_mode = 'binary')

  return image

In [ ]:
path =
train_data = preprocessingImages1(path)

In [ ]:
def preprocessingImages2(path):
  #input: Path
  #output: Pre processed images
  image_data=ImageDataGenerator(rescale=1/255)
  image=image_data.flow_from_directory(directory= path, target_size=(224,224), batch_size = 32,class_mode = 'binary')

  return image

In [ ]:
path=
test_data=preprocessingImages2(path)

In [ ]:
path=
val_data=preprocessingImages2(path)

In [ ]:
# Early stopping and model check point

from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping

es = EarlyStopping(monitor="val_accuracy", min_delta= 0.01, patience= 5, verbose=1, mode = 'auto')

# Model Check Point
mc = ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel,h5", verbos= 1, save_best_only=True , mode = 'auto')

cb = [es,mc]

#Model Training

In [ ]:
hs = model.fit_generator(generator=train_data, steps_per_epoch= 8, epochs= 30, verbose= 1, validation_data= val_data, validation_steps= 16, callbacks=cb)

In [ ]:
# Model Graphical Interpretation

h = hs.history()
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] ,c='red')

plt.title("acc vs val_acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] ,c='red')

plt.title("acc vs val_loss")
plt.show()

In [ ]:
# Model Accuracy

from keras.models import load_model

model = load_model("")

#Model Accuracy

In [ ]:
acc = model.evaluate_generator(test_data)[1]

print(f"the accuracy of out model is {acc*100}%")

In [ ]:
from keras.preprocessing.image import load_img,img_to_array

In [ ]:
path=

img = load_img(path, target_size= (224,224))
input_arr = img_to_array(img)/255

plt.imshow(imput_arr)
plt.show()

input_arr.shape

input_arr = np.expand_dims(input_arr, axis = 0)

pred = model.predict_classes(input_arr)[0][0]
pred

if(pred==0):
  print("The MRI is having a Tumor")
else:
  print("The MRI is Not having a Tumor")